In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [15]:
# Cargar los datos
data = pd.read_excel("data_customer_classification.xlsx")

In [16]:
# Convertir la columna 'trans_date' a tipo datetime
data['trans_date'] = pd.to_datetime(data['trans_date'])

# Calcular la frecuencia de compra por cliente
data['frequency'] = data.groupby('customer_id')['customer_id'].transform('count')

# Calcular el total gastado por cliente
data['total_spent'] = data.groupby('customer_id')['tran_amount'].transform('sum')

# Calcular el máximo gastado por cliente
data['max_spent'] = data.groupby('customer_id')['tran_amount'].transform('max')

In [36]:
def genCateg(x):
    if x<500:
        return 0
    elif x<1000:
        return 1
    else:
        return 2

In [38]:
data_unique

,customer_id,trans_date,tran_amount,frequency,total_spent,max_spent
0,CS5295,2013-02-11,35,25,1637,104
1,CS4768,2015-03-15,39,26,1721,105
2,CS2122,2013-02-26,52,23,1648,105
3,CS1217,2011-11-16,99,20,1431,104
4,CS1850,2013-11-20,78,24,1908,105
...,...,...,...,...,...,...
112704,CS8478,2013-05-25,56,7,287,56
112727,CS8326,2013-08-05,42,5,209,62
113504,CS8409,2011-06-20,72,8,399,72
113951,CS8479,2011-08-31,19,5,223,67


In [39]:
# Eliminar duplicados
data_unique = data.drop_duplicates(subset=['customer_id'])

# Generar etiquetas de clasificación basadas en el total gastado
data_unique['customer_category'] = list([genCateg(i) for i in data_unique['total_spent']])

# Crear un nuevo DataFrame para características y etiquetas
features_labels = data_unique[['frequency', 'total_spent', 'max_spent', 'customer_category']]

# Normalizar los datos
# scaler = StandardScaler()
# features_labels[['frequency', 'total_spent', 'max_spent']] = scaler.fit_transform(features_labels[['frequency', 'total_spent', 'max_spent']])

# Codificar las etiquetas de clasificación
label_encoder = LabelEncoder()
features_labels['customer_category'] = label_encoder.fit_transform(features_labels['customer_category'])

# Definir las características y las etiquetas
X = features_labels[['frequency', 'total_spent', 'max_spent']].values
y = features_labels['customer_category'].values

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

C:\Users\angelo\AppData\Local\Temp\ipykernel_15748\631797487.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_unique['customer_category'] = list([genCateg(i) for i in data_unique['total_spent']])
C:\Users\angelo\AppData\Local\Temp\ipykernel_15748\631797487.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_labels['customer_category'] = label_encoder.fit_transform(features_labels['customer_category'])


In [40]:
print(features_labels)

        frequency  total_spent  max_spent  customer_category
0              25         1637        104                  2
1              26         1721        105                  2
2              23         1648        105                  2
3              20         1431        104                  2
4              24         1908        105                  2
...           ...          ...        ...                ...
112704          7          287         56                  0
112727          5          209         62                  0
113504          8          399         72                  0
113951          5          223         67                  0
117419          5          164         51                  0

[6889 rows x 4 columns]


In [41]:
print(X_train[0], y_train[0])

[  20 1474  101] 2


In [43]:
from tensorflow.keras.utils import to_categorical

# Convertir y_train y y_test en representaciones one-hot
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

In [44]:
print(X_train[0], y_train_one_hot[0])

[  20 1474  101] [0. 0. 1.]


In [47]:
import numpy as np

# Suponiendo que tienes las predicciones en one-hot llamadas 'predictions_one_hot'
# y que cada fila representa una predicción para un dato

# Invertir la codificación one-hot
predictions = np.argmax(y_train_one_hot, axis=1)

# Contar la frecuencia de cada categoría
category_counts = np.bincount(predictions)

# Imprimir el resultado
for i, count in enumerate(category_counts):
    print(f"Categoría {i}: {count} datos")

Categoría 0: 483 datos
Categoría 1: 1415 datos
Categoría 2: 3613 datos


In [53]:
# Construir el modelo
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(3,)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

# Compilar el modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrenar el modelo
historial = model.fit(X_train, y_train, epochs=250, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/250
173/173 [==============================] - 2s 5ms/step - loss: 10.5560 - accuracy: 0.6478 - val_loss: 0.6145 - val_accuracy: 0.7903
Epoch 2/250
173/173 [==============================] - 1s 3ms/step - loss: 0.4383 - accuracy: 0.8503 - val_loss: 0.3193 - val_accuracy: 0.8774
Epoch 3/250
173/173 [==============================] - 1s 3ms/step - loss: 0.2759 - accuracy: 0.8913 - val_loss: 0.2281 - val_accuracy: 0.9209
Epoch 4/250
173/173 [==============================] - 0s 3ms/step - loss: 0.2557 - accuracy: 0.8886 - val_loss: 0.2323 - val_accuracy: 0.9071
Epoch 5/250
173/173 [==============================] - 1s 3ms/step - loss: 0.2081 - accuracy: 0.9142 - val_loss: 0.1917 - val_accuracy: 0.9086
Epoch 6/250
173/173 [==============================] - 1s 3ms/step - loss: 0.1849 - accuracy: 0.9249 - val_loss: 0.1550 - val_accuracy: 0.9354
Epoch 7/250
173/173 [==============================] - 0s 3ms/step - loss: 0.1699 - accuracy: 0.9318 - val_loss: 0.1766 - val_accuracy: 0.916

In [56]:
# Suponiendo que ya tienes el modelo entrenado 'model'
categ_dicc = {
    0 : "bajo valor",
    1 : "medio valor",
    2 : "alto valor" 
}

# Definir las características para predecir
new_data = [[1, 10, 20],   # Ejemplo 1
            [15, 700, 101],   # Ejemplo 2
            [20, 2000, 200]]  # Ejemplo 3


# Hacer la predicción
predictions = model.predict(new_data)

# Decodificar las predicciones
decoded_predictions = label_encoder.inverse_transform(np.argmax(predictions, axis=1))

print("Predicciones:")
for i, pred in enumerate(decoded_predictions):
    print(f"Ejemplo {i+1}: {categ_dicc[pred]}")


1/1 [==============================] - 0s 84ms/step
Predicciones:
Ejemplo 1: bajo valor
Ejemplo 2: medio valor
Ejemplo 3: alto valor
